# GRADIENT BOOSTING REGRESSOR
No ha salido muy bien

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
diamonds_train = pd.read_csv('data/diamonds_train.csv')
diamonds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  object 
 2   color    40455 non-null  object 
 3   clarity  40455 non-null  object 
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   price    40455 non-null  int64  
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.1+ MB


In [3]:
diamonds_predict = pd.read_csv('data/diamonds_test.csv')

In [4]:
diamonds_train[diamonds_train['carat']<1].describe()

,carat,depth,table,price,x,y,z
count,26155.000000,26155.000000,26155.000000,26155.000000,26155.000000,26155.000000,26155.000000
mean,0.508951,61.737041,57.193871,1625.447639,5.042552,5.050535,3.115290
std,0.198101,1.327113,2.212901,1114.414551,0.642784,0.640815,0.400528
min,0.200000,52.300000,44.000000,326.000000,0.000000,0.000000,0.000000
25%,0.330000,61.100000,56.000000,757.000000,4.440000,4.450000,2.740000
50%,0.460000,61.800000,57.000000,1179.000000,4.950000,4.950000,3.050000
75%,0.700000,62.500000,58.000000,2304.000000,5.660000,5.660000,3.490000
max,0.990000,79.000000,79.000000,9636.000000,6.820000,6.740000,5.060000


In [5]:
diamonds_train[(diamonds_train['carat']>1) & (diamonds_train['carat']<2)].describe()

,carat,depth,table,price,x,y,z
count,11512.000000,11512.000000,11512.000000,11512.000000,11512.000000,11512.000000,11512.000000
mean,1.242681,61.754656,57.779960,7478.653579,6.867794,6.861926,4.236100
std,0.226771,1.477758,2.086285,3178.403881,0.425233,0.419165,0.278123
min,1.010000,43.000000,43.000000,2017.000000,0.000000,0.000000,0.000000
25%,1.040000,61.000000,56.000000,5018.750000,6.520000,6.520000,4.030000
50%,1.190000,61.900000,58.000000,6577.000000,6.770000,6.760000,4.180000
75%,1.500000,62.600000,59.000000,9281.250000,7.240000,7.230000,4.470000
max,1.990000,71.800000,70.000000,18806.000000,8.280000,8.170000,5.120000


In [6]:
diamonds_train[diamonds_train['carat']>=2].describe()

,carat,depth,table,price,x,y,z
count,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000
mean,2.135618,61.824346,58.324102,14816.021302,8.233177,8.240043,5.068503
std,0.223455,1.885183,2.417542,2782.800988,0.354667,1.298732,0.366946
min,2.000000,55.300000,50.000000,5051.000000,0.000000,0.000000,0.000000
25%,2.010000,60.700000,57.000000,13064.000000,8.060000,8.050000,4.980000
50%,2.040000,61.900000,58.000000,15245.000000,8.180000,8.170000,5.050000
75%,2.180000,62.700000,60.000000,17024.000000,8.360000,8.340000,5.150000
max,4.500000,70.600000,95.000000,18823.000000,10.230000,58.900000,8.060000


In [7]:
#linear_vars = diamonds_train.select_dtypes(include=[np.number]).columns
#diamonds_train

## GETTING RID OF 0 IN xyz

### TRAIN

In [8]:
diamonds_train = diamonds_train[(diamonds_train['x'] != 0) & (diamonds_train['y'] != 0) & (diamonds_train['z'] != 0)]
diamonds_train.shape

(40439, 10)

### TEST

Celda no ejecutada porque al eliminar rows en el csv de test, da fallo la submission

diamonds_predict = diamonds_predict[(diamonds_predict['x'] != 0) & (diamonds_predict['y'] != 0) & (diamonds_predict['z'] != 0)]
diamonds_predict.info()

## GET RID OF OUTLIERS

Tried with 75-25, not a good idea shape=(6465, 10), we gonna try with some less outliers

In [9]:
# Hay un diamante con un depth de 79
diamonds_train = diamonds_train[(diamonds_train["depth"]<71) & (diamonds_train["depth"]>54)]

# Hay 3 mayores de 75 y dos menores de 45
diamonds_train = diamonds_train[(diamonds_train["table"]<70) & (diamonds_train["table"]>52)]

diamonds_train = diamonds_train[(diamonds_train["x"]<10)]  
#solo dropeamos las que sean x<10 porque coincide con 'y' en sus dos únicos valores por encima de 10 y no dropeamos nada de z porque hay bastantes valores en torno a 6

diamonds_train = diamonds_train[diamonds_train['carat'] < 2.7]

In [10]:
len(diamonds_train[diamonds_train['carat'] > 2.7])

0

In [11]:
diamonds_train.shape

(40312, 10)

## VOLUME COL

In [12]:
diamonds_train['volume'] = diamonds_train.x * diamonds_train.y * diamonds_train.z
diamonds_predict['volume'] = diamonds_predict.x * diamonds_predict.y * diamonds_predict.z

In [13]:
diamonds_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       13485 non-null  int64  
 1   carat    13485 non-null  float64
 2   cut      13485 non-null  object 
 3   color    13485 non-null  object 
 4   clarity  13485 non-null  object 
 5   depth    13485 non-null  float64
 6   table    13485 non-null  float64
 7   x        13485 non-null  float64
 8   y        13485 non-null  float64
 9   z        13485 non-null  float64
 10  volume   13485 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 1.1+ MB


## l/w ratio (x/y)

In [14]:
diamonds_train['l/w ratio'] = diamonds_train.x/diamonds_train.y
diamonds_predict['l/w ratio'] = diamonds_predict.x/diamonds_predict.y

In [15]:
diamonds_train['l/w ratio'].describe()

count    40312.000000
mean         0.999369
std          0.011200
min          0.137351
25%          0.992593
50%          0.995717
75%          1.006920
max          1.615572
Name: l/w ratio, dtype: float64

## depth

In [16]:
diamonds_train['depth ratio'] = diamonds_train.z/diamonds_train.y
diamonds_predict['depth ratio'] = diamonds_predict.z/diamonds_predict.y

## log carat

In [17]:
diamonds_train['carat log'] = np.log(diamonds_train['carat'])
diamonds_predict['carat log'] = np.log(diamonds_predict['carat'])

## density

In [19]:
#diamonds_train['density'] = diamonds_train['carat']/diamonds_train['volume']
#diamonds_predict['density'] = diamonds_predict['carat']/diamonds_predict['volume']

In [ ]:
#diamonds_train = diamonds_train.replace([np.inf, -np.inf, 0], np.nan)
#diamonds_predict = diamonds_predict.replace([np.inf, -np.inf, 0], np.nan)

In [20]:
#diamonds_train.isnull().sum()

carat          0
cut            0
color          0
clarity        0
depth          0
table          0
price          0
x              0
y              0
z              0
volume         0
l/w ratio      0
depth ratio    0
carat log      0
dtype: int64

In [ ]:
#diamonds_train.dropna(inplace=True)
#diamonds_predict.dropna(inplace=True)

# ENCODING CATEGORIES

## TRAIN

In [21]:
cut_num = {'Ideal': 5, 'Premium': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1}
diamonds_train['cut_num'] = diamonds_train['cut'].replace(cut_num)

In [22]:
color_num = {'D': 7, 'E': 6, 'F': 5, 'G': 4, 'H': 3, 'I': 2, 'J': 1}
diamonds_train['color_num'] = diamonds_train['color'].replace(color_num)

In [23]:
clarity_num = {'IF': 8, 'VVS1': 7, 'VVS2': 6, 'VS1': 5, 'VS2': 4, 'SI1': 3, 'SI2': 2, 'I1': 1}
diamonds_train['clarity_num'] = diamonds_train['clarity'].replace(clarity_num)

## TEST CSV (nuestro diamonds_predict)

In [24]:

diamonds_predict['cut_num'] = diamonds_predict['cut'].replace(cut_num)

diamonds_predict['color_num'] = diamonds_predict['color'].replace(color_num)

diamonds_predict['clarity_num'] = diamonds_predict['clarity'].replace(clarity_num)

# CATEGORIES/CARAT

## CUT

In [25]:
diamonds_train['cut/crt'] = diamonds_train['cut_num']/diamonds_train['carat']
diamonds_predict['cut/crt'] = diamonds_predict['cut_num']/diamonds_predict['carat']

## COLOR

In [26]:
diamonds_train['color/crt'] = diamonds_train['color_num']/diamonds_train['carat']
diamonds_predict['color/crt'] = diamonds_predict['color_num']/diamonds_predict['carat']

## CLARITY

In [27]:
diamonds_train['clarity/crt'] = diamonds_train['clarity_num']/diamonds_train['carat']
diamonds_predict['clarity/crt'] = diamonds_predict['clarity_num']/diamonds_predict['carat']

# FEATURE ENGINERING

In [28]:
diamonds_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13485 non-null  int64  
 1   carat        13485 non-null  float64
 2   cut          13485 non-null  object 
 3   color        13485 non-null  object 
 4   clarity      13485 non-null  object 
 5   depth        13485 non-null  float64
 6   table        13485 non-null  float64
 7   x            13485 non-null  float64
 8   y            13485 non-null  float64
 9   z            13485 non-null  float64
 10  volume       13485 non-null  float64
 11  l/w ratio    13482 non-null  float64
 12  depth ratio  13482 non-null  float64
 13  carat log    13485 non-null  float64
 14  cut_num      13485 non-null  int64  
 15  color_num    13485 non-null  int64  
 16  clarity_num  13485 non-null  int64  
 17  cut/crt      13485 non-null  float64
 18  color/crt    13485 non-null  float64
 19  clar

### Train

In [ ]:
#diamonds_train['price/carat'] = (diamonds_train['price']/diamonds_train['carat']).round(2)

## HEATMAP

Depth & cut no están muy relacionados con el precio

In [ ]:
plt.figure(figsize=(12,12))
corr = diamonds_train.corr()
sns.heatmap(data=corr, annot=True, cbar=True)

In [ ]:
diamonds_train.groupby('cut')[['x', 'y', 'z', 'table']].mean()

In [ ]:
diamonds_predict.info()

In [ ]:
diamonds_train.info()

# ESCALAR

In [54]:
#cat_features = ['cut', 'color', 'clarity']
features = ['carat', 'l/w ratio', 'volume', 'cut_num', 'color_num', 'clarity_num']
#predict_feat = ['id','carat', 'table', 'depth', 'x', 'y', 'z', 'cut_num', 'color_num', 'clarity_num']
target = 'price'

### TRAIN

In [30]:
from sklearn.preprocessing import StandardScaler

In [55]:
scaler = StandardScaler()

In [56]:
X = scaler.fit_transform(diamonds_train[features])
y = diamonds_train[target]

### TEST CSV (PREDICT)

In [57]:
X_pred = scaler.fit_transform(diamonds_predict[features])

In [36]:
diamonds_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13485 non-null  int64  
 1   carat        13485 non-null  float64
 2   cut          13485 non-null  object 
 3   color        13485 non-null  object 
 4   clarity      13485 non-null  object 
 5   depth        13485 non-null  float64
 6   table        13485 non-null  float64
 7   x            13485 non-null  float64
 8   y            13485 non-null  float64
 9   z            13485 non-null  float64
 10  volume       13485 non-null  float64
 11  l/w ratio    13482 non-null  float64
 12  depth ratio  13482 non-null  float64
 13  carat log    13485 non-null  float64
 14  cut_num      13485 non-null  int64  
 15  color_num    13485 non-null  int64  
 16  clarity_num  13485 non-null  int64  
 17  cut/crt      13485 non-null  float64
 18  color/crt    13485 non-null  float64
 19  clar

# TRAIN-TEST-SPLIT

In [37]:
from sklearn.model_selection import train_test_split 

In [ ]:
#X = diamonds_train[features]
#y = diamonds_train[target]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

### TEST

# MODELS

In [39]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score

## 1. RANDOM FOREST

In [59]:
rfr_model = RandomForestRegressor(n_estimators=300)

In [60]:
rfr_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300)

In [61]:
y_pred = rfr_model.predict(X_test)

In [62]:
cv_score = cross_val_score(estimator=rfr_model, X=X_train, y=y_train, cv=5,verbose = 1)
print('Cross validation score : %.4f' % rfr_model.score(X_test, y_test))
print(cv_score)
mae = mean_absolute_error(y_test,y_pred)
print("mae: %f" %(mae))
r2 = rfr_model.score(X_test,y_test)
print("R2: %f" %(r2))
mse = mean_squared_error(y_test,y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test,y_pred))**0.5
print("rmse: %f" %(rmse))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


Cross validation score : 0.9823
[0.98074232 0.98151686 0.97980522 0.98160776 0.98085216]
mae: 261.619864
R2: 0.982268
mse: 279160.762988
rmse: 528.356663


In [ ]:
# volume without xyz
Cross validation score : 0.9819
[0.98231093 0.98064966 0.98158205 0.97886279 0.98079611]
mae: 267.725271
R2: 0.981928
mse: 289137.876141
rmse: 537.715423
"""
Cross validation score : 0.981
[0.98114748 0.97993092 0.97976217 0.9798617  0.98153202]
mae: 283.627434
R2: 0.980543
mse: 314089.953265
rmse: 560.437288
"""

In [ ]:
#crossval_score = cross_val_score(rfr_model, X_train, y_train, cv=10)
#print(cross_val_score)

## DECISION TREE

In [ ]:
dtr_model = DecisionTreeRegressor(random_state = 0) #rstate=0 porque los resultados son mejores que sin poner nada o poniendo 1
dtr_model.fit(X_train, y_train)
y_pred = dtr_model.predict(X_test)

In [ ]:
cv_score = cross_val_score(estimator=dtr_model, X=X_train, y=y_train, cv=5,verbose = 1)
print('Cross validation score : %.3f' % dtr_model.score(X_test, y_test))
print(cv_score)
mae = mean_absolute_error(y_test,y_pred)
print("mae: %f" %(mae))
r2 = dtr_model.score(X_test,y_test)
print("R2: %f" %(r2))
mse = mean_squared_error(y_test,y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test,y_pred))**0.5
print("rmse: %f" %(rmse))


mae = mean_absolute_error(y_test,y_pred)
print("mae: %f" %(mae))
r2 = rfr_model.score(X_test,y_test)
print("R2: %f" %(r2))
mse = mean_squared_error(y_test,y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test,y_pred))**0.5
print("rmse: %f" %(rmse))

RESULTADOS SIN CAMBIAR RANDOM STATE:, con random state = 0, mejora
mae: 372.840853
R2: 0.980116
mse: 579384.121723
rmse: 761.172859

## GRADIENT BOOSTING

In [63]:
gbr_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=7, loss='ls',verbose = 10)
#max_depth 1, mal, 2, mal, 4 ok (553 rmse), 8 ok (542)

In [64]:
gbr_model.fit(X_train , y_train)
y_pred = gbr_model.predict(X_test)

      Iter       Train Loss   Remaining Time 
         1    12928716.9000           18.55s
         2    10567420.4358           17.21s
         3     8651989.1387           16.23s
         4     7096486.6262           15.62s
         5     5831053.7898           14.80s
         6     4802826.1258           14.27s
         7     3966616.7416           14.15s
         8     3285800.3941           13.80s
         9     2729944.3593           13.52s
        10     2277509.9917           13.29s
        11     1906858.2769           13.21s
        12     1603804.8894           13.02s
        13     1356979.6734           12.85s
        14     1157179.9499           12.69s
        15      993314.1314           12.67s
        16      858175.2572           12.54s
        17      747175.8383           12.42s
        18      655975.5944           12.29s
        19      581265.7798           12.34s
        20      520263.4343           12.22s
        21      469441.6519           12.10s
        2

In [65]:
cv_score = cross_val_score(estimator=gbr_model, X=X_train, y=y_train, cv=5,verbose = 1)
print('Cross validation score : %.3f' % gbr_model.score(X_test, y_test))
print(cv_score)
mae = mean_absolute_error(y_test,y_pred)
print("mae: %f" %(mae))
r2 = gbr_model.score(X_test,y_test)
print("R2: %f" %(r2))
mse = mean_squared_error(y_test,y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test, y_pred))**0.5
print("rmse: %f" %(rmse))

      Iter       Train Loss   Remaining Time 
         1    13000928.4729           11.88s
         2    10627300.7402           11.39s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


         3     8698787.6998           10.95s
         4     7133544.0533           11.03s
         5     5859702.1017           11.22s
         6     4822141.5726           10.90s
         7     3981576.3777           10.85s
         8     3293402.8912           11.08s
         9     2734591.2212           10.99s
        10     2280510.0796           10.78s
        11     1910404.4436           10.61s
        12     1606747.7090           10.61s
        13     1358598.1854           10.66s
        14     1157415.2721           10.52s
        15      992225.7363           10.39s
        16      856347.8165           10.35s
        17      744895.9553           10.28s
        18      654219.8272           10.17s
        19      579466.4723           10.13s
        20      517151.2790           10.12s
        21      465224.6027           10.08s
        22      422821.8674            9.97s
        23      387646.6399            9.88s
        24      358095.2188            9.83s
        25

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   58.2s finished


In [ ]:
n_estimators=100, depth=8
Cross validation score : 0.981
[0.98324256 0.9826404  0.9822319  0.97931829 0.98165835]
mae: 264.090517
R2: 0.982064
mse: 286951.932481
rmse: 535.678945

Cross validation score : 0.982
[0.98290342 0.98162418 0.98218219 0.97854537 0.98101767]
mae: 266.467269
R2: 0.982228
mse: 284328.172392
rmse: 533.224317
max_depth = 200 
rmse: 531.976353

## LIGHTGBM

In [40]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [41]:
hgbr_model = HistGradientBoostingRegressor(max_depth=10, max_iter=200)

In [42]:
hgbr_model.fit(X_train, y_train)
y_pred = hgbr_model.predict(X_test)

In [43]:
cv_score = cross_val_score(estimator=hgbr_model, X=X_train, y=y_train, cv=5,verbose = 1)
print('Cross validation score : %.4f' % hgbr_model.score(X_test, y_test))
print(cv_score)
mae = mean_absolute_error(y_test, y_pred)
print("mae: %f" %(mae))
r2 = hgbr_model.score(X_test,y_pred)
print("R2: %.4f" %(r2))
mse = mean_squared_error(y_test, y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test, y_pred))**0.5
print("rmse: %f" %(rmse))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Cross validation score : 0.9837
[0.98263537 0.98269    0.98213042 0.98366576 0.9816379 ]
mae: 258.030845
R2: 1.0000
mse: 256072.135040
rmse: 506.035705


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.9s finished


In [ ]:
# COn l/w y sinn nada 
Cross validation score : 0.9822
[0.98363749 0.98244627 0.98260417 0.98031694 0.98236592]
mae: 277.914486
R2: 1.0000
mse: 285311.164230
rmse: 534.145265
#max iter =200
Cross validation score : 0.9824
[0.98266068 0.98168035 0.98244195 0.97939003 0.98154611]
mae: 277.697224
R2: 1.0000
mse: 281983.359959
rmse: 531.021054
#max_iter =300
Cross validation score : 0.9823
[0.98247221 0.98136899 0.98257238 0.97945764 0.98136524]
mae: 274.923396
R2: 1.0000
mse: 282725.056504
rmse: 531.718964

#max iter = 200
Cross validation score : 0.9822
[0.98283749 0.98146759 0.98250352 0.97946861 0.98103671]
mae: 279.372035
R2: 1.0000
mse: 285130.738400
rmse: 533.976346

## EXTRA TREES REGRESSOR

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
etr_model = ExtraTreesRegressor()
etr_model.fit(X_train, y_train)
y_pred = etr_model.predict(X_test)

In [ ]:
cv_score = cross_val_score(estimator=etr_model, X=X_train, y=y_train, cv=5,verbose = 1)
print('Cross validation score : %.4f' % etr_model.score(X_test, y_test))
print(cv_score)
mae = mean_absolute_error(y_test, y_pred)
print("mae: %f" %(mae))
r2 = etr_model.score(X_test,y_pred)
print("R2: %.4f" %(r2))
mse = mean_squared_error(y_test, y_pred)
print("mse: %f" %(mse))
rmse = (mean_squared_error(y_test, y_pred))**0.5
print("rmse: %f" %(rmse))

## PREDICTION

In [66]:
X_predict = X_pred
predictions = gbr_model.predict(X_predict)
diamonds_id = diamonds_predict['id']

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [46]:
diamonds_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13485 non-null  int64  
 1   carat        13485 non-null  float64
 2   cut          13485 non-null  object 
 3   color        13485 non-null  object 
 4   clarity      13485 non-null  object 
 5   depth        13485 non-null  float64
 6   table        13485 non-null  float64
 7   x            13485 non-null  float64
 8   y            13485 non-null  float64
 9   z            13485 non-null  float64
 10  volume       13485 non-null  float64
 11  l/w ratio    13482 non-null  float64
 12  depth ratio  13482 non-null  float64
 13  carat log    13485 non-null  float64
 14  cut_num      13485 non-null  int64  
 15  color_num    13485 non-null  int64  
 16  clarity_num  13485 non-null  int64  
 17  cut/crt      13485 non-null  float64
 18  color/crt    13485 non-null  float64
 19  clar

# GENERAR DF

In [47]:
submission = pd.DataFrame({'id': diamonds_id, 'price': predictions})

In [48]:
submission.shape

(13485, 2)

In [ ]:
submission['id'] = submission['id'].astype('int64')

In [49]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      13485 non-null  int64  
 1   price   13485 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 210.8 KB


In [50]:
submission.to_csv('submission.csv', index=False)